# Overview

Self-supervised learning(SSL) has emerged as powerful approach to extract mearninggul representations from data without requiring extensive labeled datasets. And several self-supervised learning techniques have been successfully applied to time series data. For example:

### 1.Time constractive learning(TCL)

In Time COntrastive Learning(TCL), the goal is to learn representattions that can differetiate between time series segments from different time steps. This can be done using contrastive learning methods, where positive examples come from the same time series, and negative examples are take from different time series or time steps.

### 2.Temporal jigsaw

In this approach, a time series is divided into several non-overlapping segments, and the model's task is to predict the correct ordering of these segments.

### 3.Future prediction(auto-regressive modeling)

The model predicts future time steps based on the past. This can be done in an auto-regressive manner, the model uses its predictions to forecast subsequent time steps.

### 4.Masked reconstruction

It is masking certain time steps and training the model to reconstruct the missing data.

### 5.Time shift detection

The model's pretext task is to detect whether a given time series segment has been shifted(ot lagged) in time. The model learning to recognize temporal inconsistencies, the model captures temporal dynamics and patterns.


Below is an example of implementing contrastive learning for time series representation using PyTorch. 

# Data preparation

In [1]:
import torch
import numpy as np

def generate_sine_wave(n_series, length, noise_factor=0.1):
    x=np.linspace(0,4*np.pi, length)
    data=[np.sin(x)+noise_factor*np.random.rand(length) for _ in range(n_series)]
    return np.array(data)

# parameters
n_series=1000
length=50
data=generate_sine_wave(n_series, length)

In [2]:
# visualization
import plotly.graph_objects as go

fig=go.Figure()

for i in range(n_series):
    fig.add_trace(go.Scatter(
        x=np.arange(length),
        y=data[i],
        mode='lines',
        name=f'Series {i+1}',
        line=dict(width=1)
    ))
    
fig.update_layout(
    title="Synthetic Sine Wave Time Series",
    xaxis_title="Time Steps",
    yaxis_title="Amplitude",
    showlegend=False
)

fig.show()

# Define the contrastive learning model

We will define a simple contrastive learning model that use an encoder to extract features from the time series data, follwed by a contrastive loss function to learning representations.

In [3]:
import torch.nn as nn
import torch.optim as optim

class TimeSeriesEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TimeSeriesEncoder, self).__init__()
        self.lstm=nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc=nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        _, (hn,x)=self.lstm(x)
        out=self.fc(hn[-1])
        return out

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss,self).__init__()
        self.margin=margin
        
    def forward(self, positive_distances, negative_distances):
        return torch.mean(positive_distances-negative_distances+self.margin)
    

# Parameters
input_size=1
hidden_size=32
output_size=16

# Initialize encoder and loss function
encoder=TimeSeriesEncoder(input_size, hidden_size, output_size)

encoder.to("cuda")

contrastive_loss=ContrastiveLoss()
optimizer=optim.Adam(encoder.parameters(), lr=0.001)

# Training the self-supervised model

We will use pairs of positive and negative time series samples for training. Positive pairs come from the same time series, while nagtive pairs come from different series.

In [4]:
def train_contrastive_model(data, encoder, contrastive_loss, optimizer, epochs=50):
    encoder.train()
    for epoch in range(epochs):
        epoch_loss=0.0
        for i in range(len(data)-2):
            x_pos=torch.tensor(data[i:i+2,:], dtype=torch.float32).unsqueeze(-1).to("cuda")
            x_neg=torch.tensor([data[i], data[i+2]], dtype=torch.float32).unsqueeze(-1).to("cuda")
            
            # encode positive and negative pairs
            pos_features=encoder(x_pos)
            neg_features=encoder(x_neg)
            
            # compute distances
            pos_distances=torch.norm(pos_features[0]-pos_features[1])
            neg_distances=torch.norm(neg_features[0]-neg_features[1])
            
            # compute loss and backpropagate
            loss=contrastive_loss(pos_distances, neg_distances)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss+=loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], loss: {epoch_loss/len(data)}")
        
train_contrastive_model(data, encoder, contrastive_loss, optimizer)

/tmp/ipykernel_24/2381253024.py:7: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)



Epoch [1/50], loss: 0.9979871985912323
Epoch [2/50], loss: 0.9979206029772758
Epoch [3/50], loss: 0.9978254146575928
Epoch [4/50], loss: 0.9976825208663941
Epoch [5/50], loss: 0.9975507243275642
Epoch [6/50], loss: 0.9973216859698295
Epoch [7/50], loss: 0.9969788146615028
Epoch [8/50], loss: 0.9966235462427139
Epoch [9/50], loss: 0.9961566504836082
Epoch [10/50], loss: 0.9954546232223511
Epoch [11/50], loss: 0.9948247873783111
Epoch [12/50], loss: 0.9911427330374718
Epoch [13/50], loss: 0.9952281713485718
Epoch [14/50], loss: 0.9948156219124794
Epoch [15/50], loss: 0.9761860229372978
Epoch [16/50], loss: 0.9929250264167786
Epoch [17/50], loss: 0.9718932777047157
Epoch [18/50], loss: 0.9864596421122551
Epoch [19/50], loss: 0.9728497073650361
Epoch [20/50], loss: 0.9868353369235993
Epoch [21/50], loss: 0.9700736127495766
Epoch [22/50], loss: 0.9878779944181443
Epoch [23/50], loss: 0.9872099578380584
Epoch [24/50], loss: 0.9921008113026619
Epoch [25/50], loss: 0.9898220742344856
Epoch [26

# Acknowledgement

* https://blog.gopenai.com/learning-representations-for-time-series-using-self-supervised-learning-4ab2c00f670c